In [59]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils

from Unet import UNet
from utilities import mse_loss
from evaluate import reverse_diffusion, transform_img

from torchmetrics.image.inception import InceptionScore

from tqdm import tqdm


/Users/tanaytuncer/.local/share/virtualenvs/tanaytuncer-3yiFbWzj/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

In [ ]:
train_data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
from torch.utils.data import Subset

n = 2000
n_batch = 64

T = 20

subset = Subset(train_data, range(n))

X_train_loader = DataLoader(subset, batch_size=n_batch, shuffle=True, drop_last=True)

data = iter(X_train_loader)
images, labels = next(data)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5   
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

fig = plt.figure(figsize=(6, 6))
for i in range(9):
    ax = fig.add_subplot(3, 3, i + 1, xticks=[], yticks=[])
    imshow(images[i])

plt.show()

In [ ]:
print(type(images), images.shape)
print(type(labels), labels.shape)

In [ ]:
model = UNet(img_dim = 3)

print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
model

In [ ]:
model_path = '/Users/tanaytuncer/Desktop/diffusion_model/models/240124_model_cosine_res.pth'

model = UNet()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

In [ ]:
model_path

In [ ]:
inception_model = InceptionScore()

image_shape = (3, 32, 32)

model.eval()

for i in tqdm(range(100)):

  with torch.no_grad():
    generated_image = reverse_diffusion(model, n_batch, T, image_shape)

generated_image = generated_image.squeeze(0)
generated_image = (generated_image + 1) / 2
generated_image = torch.clamp(generated_image, 0, 1)

transformed_images = transform_img(generated_image)
inception_model.update(transformed_images)

avg, std = inception_model.compute()

avg, std 

#Linear: (tensor(1.), tensor(1.9868e-08))
#Cosine: (tensor(1.), tensor(1.9868e-08))